<div style="text-align: center;">
  <h2><strong></strong></h2>
</div>

<center>

## **Prueba Técnica Novaventa**
## **Staging Zone**

**Autor:** Sebastian Carvalho Salazar  

</center>

## Librerias

In [28]:
import numpy as np
import pandas as pd
from datetime import date

## Rutas

In [29]:
import yaml
import os

# Obtener la ruta absoluta del archivo paths.yaml
current_dir = current_dir = os.getcwd()
config_path = os.path.join(current_dir, '..', 'Config', 'paths.yaml')

# Cargar el contenido del archivo YAML
with open(config_path, 'r') as file:
    paths = yaml.safe_load(file)

## Datos

In [30]:
path_1 = paths['RawZone']['dataset_1']
dataset_1 = pd.read_csv(path_1)

path_2 = paths['RawZone']['dataset_2']
dataset_2 = pd.read_csv(path_2)

productos = dataset_1[['COD_PRODUCTO',	'CATEGORIA']].drop_duplicates(subset=['COD_PRODUCTO'])

In [31]:
print(f"* Numero de (Registros,Columnas) en Dataset Sample 1: {dataset_1.shape}")
print(f"    - Numero de clientes: {len(dataset_1.index.unique())}")
print(f"* Numero de (Registros,Columnas) en Dataset Sample 2: {dataset_2.shape}")
print(f"    - Numero de clientes: {len(dataset_2.index.unique())}")

* Numero de (Registros,Columnas) en Dataset Sample 1: (231000, 7)
    - Numero de clientes: 231000
* Numero de (Registros,Columnas) en Dataset Sample 2: (231000, 5)
    - Numero de clientes: 231000


In [32]:
dataset_1.isna().sum()

UUID_CLIENTE_CONSUMIDOR       0
PEDIDO                        0
FECHA_SOLUCION                0
COD_PRODUCTO                  0
CATEGORIA                  1042
UNIDADES_BRUTAS               0
VENTA_BRUTA_CON_IVA           0
dtype: int64

In [33]:
# Llenar los valores NaN con el primer valor no nulo del grupo
dataset_1['CATEGORIA'] = dataset_1.groupby('COD_PRODUCTO')['CATEGORIA'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
# Si aún hay valores NaN, llenarlos con 'Desconocido'
dataset_1['CATEGORIA'] = dataset_1['CATEGORIA'].fillna('Desconocido')

In [34]:
dataset_1.isna().sum()

UUID_CLIENTE_CONSUMIDOR    0
PEDIDO                     0
FECHA_SOLUCION             0
COD_PRODUCTO               0
CATEGORIA                  0
UNIDADES_BRUTAS            0
VENTA_BRUTA_CON_IVA        0
dtype: int64

In [35]:
dataset_2.isna().sum()

UUID_CLIENTE_CONSUMIDOR            0
DEPARTAMENTO                      71
CIUDAD                            71
GENERO_DIM_CLIENTE             33978
FECHANACIMIENTO_DIM_CLIENTE    29441
dtype: int64

In [36]:
# Llenar los valores NaN con el primer valor no nulo del grupo
dataset_2['CIUDAD'] = dataset_2.groupby('UUID_CLIENTE_CONSUMIDOR')['CIUDAD'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
dataset_2['DEPARTAMENTO'] = dataset_2.groupby('UUID_CLIENTE_CONSUMIDOR')['DEPARTAMENTO'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
dataset_2['GENERO_DIM_CLIENTE'] = dataset_2.groupby('UUID_CLIENTE_CONSUMIDOR')['GENERO_DIM_CLIENTE'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
dataset_2['FECHANACIMIENTO_DIM_CLIENTE'] = dataset_2.groupby('UUID_CLIENTE_CONSUMIDOR')['FECHANACIMIENTO_DIM_CLIENTE'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
# Si aún hay valores NaN, llenarlos con 'Desconocido'
dataset_2['CIUDAD'] = dataset_2['CIUDAD'].fillna('Desconocido')
dataset_2['DEPARTAMENTO'] = dataset_2['DEPARTAMENTO'].fillna('Desconocido')
dataset_2['GENERO_DIM_CLIENTE'] = dataset_2['GENERO_DIM_CLIENTE'].fillna('Desconocido')
# dataset_2['FECHANACIMIENTO_DIM_CLIENTE'] = dataset_2['FECHANACIMIENTO_DIM_CLIENTE'].fillna('Desconocido')

In [37]:
dataset_2.isna().sum()

UUID_CLIENTE_CONSUMIDOR            0
DEPARTAMENTO                       0
CIUDAD                             0
GENERO_DIM_CLIENTE                 0
FECHANACIMIENTO_DIM_CLIENTE    29441
dtype: int64

## Transformación

In [38]:
# Agregar la frecuencia de compra
dataset_1['FREQUENCIA_COMPRA'] = dataset_1.groupby(['UUID_CLIENTE_CONSUMIDOR', 'COD_PRODUCTO'])['COD_PRODUCTO'].transform('count')

# Crear la matriz de usuarios contra productos basada en la frecuencia de compra
user_item_matrix = dataset_1.pivot_table(index='UUID_CLIENTE_CONSUMIDOR', columns='COD_PRODUCTO', values='FREQUENCIA_COMPRA', aggfunc='sum').fillna(0)

In [39]:
user_item_matrix.head()

COD_PRODUCTO,2,10,19,24,25,64,65,70,82,87,...,97994,97995,97996,97997,97998,98034,98108,98130,98204,342421
UUID_CLIENTE_CONSUMIDOR,,,,,,,,,,,,,,,,,,,,,
3232A74E9E5AC632B67F000001615250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3232A86CD5286AA5E17F000001615250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3232AC8F6D324E487D7F000001615250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3232AD9F1B20EF57907F000001615250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3232AECD7274E2EDF37F000001615250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
dataset_2['FECHANACIMIENTO_DIM_CLIENTE'] = pd.to_datetime(dataset_2['FECHANACIMIENTO_DIM_CLIENTE']).dt.date

def calculate_age(birthdate):
    today = date.today()
    return today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))

dataset_2['EDAD'] = dataset_2['FECHANACIMIENTO_DIM_CLIENTE'].apply(calculate_age)

dataset_2.head()

,UUID_CLIENTE_CONSUMIDOR,DEPARTAMENTO,CIUDAD,GENERO_DIM_CLIENTE,FECHANACIMIENTO_DIM_CLIENTE,EDAD
0,5F333C92C61098CC840A180313615250,ANTIOQUIA,MEDELLÍN,F,1984-05-03,40.0
1,323C3C3B1404F866097F000001615250,VALLE DEL CAUCA,CALI,F,1972-12-26,51.0
2,6C2FA988251C4F35BD0A180313615250,VALLE DEL CAUCA,CALI,F,1991-11-18,32.0
3,37908B3B6309B1549E7F000001615250,NORTE DE SANTANDER,PAMPLONA,F,1983-07-16,40.0
4,85C5EF6E09B085614D0A180327615250,"BOGOTÁ, D.C.","BOGOTÁ, D.C.",Desconocido,NaT,NaN


## Guardar

In [41]:
dataset_1.to_csv(paths['StagingZone']['dataset_1'], sep=";")
dataset_2.to_csv(paths['StagingZone']['dataset_2'], sep=";")
productos.to_csv(paths['StagingZone']['productos'], sep=";")
user_item_matrix.to_parquet(paths['StagingZone']['user_item_matrix'])

# FIN
---